https://github.com/vslobody/MUSIC/blob/master/music.py
https://github.com/dengjunquan/DoA-Estimation-MUSIC-ESPRIT


In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy.constants import speed_of_light
import matplotlib.pyplot as plt
from typing import Tuple
from ipywidgets import interact, BoundedIntText


In [3]:
INCIDENT_ANGLE = -50  # incident signal angle in degrees
ANTENNA_DISTANCE = 6e-2  # distance between antennas in m
FREQUENCY_DEVIATION = 250e3
NUM_ANTENNAS = 2
BLUETOOTH_FREQUENCY = 2450e6  # Center frequency of Bluetooth channel in Hz
CTE_FREQUENCY = BLUETOOTH_FREQUENCY + FREQUENCY_DEVIATION
CTE_WAVELENGTH = speed_of_light / CTE_FREQUENCY  # Bluetooth wavelength in m
CTE_TIME = 160e-6  # Constant Tone Extension time
CTE_SAMPLES = 50
NUM_SOURCES = 1
NUM_ANGLES_TO_SEARCH = 360
MAX_SEARCH_ANGLE = 90
SNR = 10


In [4]:
def response_vec(num_antennas: int, incident_angle: float) -> np.ndarray:
    """Expected antenna array response column vector of IQ values, not accounting for noise"""
    antennas = np.arange(0, num_antennas) * ANTENNA_DISTANCE
    steering = np.exp(
        2j * np.pi * antennas * np.sin(np.deg2rad(incident_angle)) / CTE_WAVELENGTH
    ).reshape((-1, 1))
    return steering


In [5]:
def sample_cov(antenna_response: np.ndarray) -> np.ndarray:
    """Returns the sample covariance matrix for the given antenna response"""
    sample_times = np.linspace(0, CTE_TIME, CTE_SAMPLES)
    num_antennas = antenna_response.size

    cov = np.zeros(shape=(num_antennas, num_antennas), dtype=complex)
    for t in sample_times:
        iq_sample_at_t = antenna_response + np.sqrt(0.5 / SNR) * np.random.randn(
            num_antennas
        )
        cov += iq_sample_at_t @ iq_sample_at_t.conjugate().T
    cov /= CTE_SAMPLES

    return cov


In [6]:
def music(covariance: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Perform the MUSIC algorithm for the given covariance matrix.
    Returns the angles searched, with their corresponding power"""
    num_antennas = covariance.shape[0]
    _, cov_eigvecs = np.linalg.eig(covariance)
    non_noise_cov_eigvecs = cov_eigvecs[:, NUM_SOURCES:num_antennas]
    search_angles = np.linspace(
        -MAX_SEARCH_ANGLE, MAX_SEARCH_ANGLE, NUM_ANGLES_TO_SEARCH
    )

    signal_powers = np.ndarray(shape=(NUM_ANGLES_TO_SEARCH,), dtype=complex)
    for idx, angle in enumerate(search_angles):
        steering_at_test_angle = response_vec(num_antennas, angle)
        signal_powers[idx] = (
            1
            / (
                steering_at_test_angle.conjugate().T
                @ non_noise_cov_eigvecs
                @ non_noise_cov_eigvecs.conjugate().T
                @ steering_at_test_angle
            )
        ).item()

    return search_angles, signal_powers


In [7]:
np.random.seed(0)
antennas_input = BoundedIntText(value=5, min=2, description="Antennas:")


@interact(num_antennas=antennas_input)
def draw_pseudo_spectrum(num_antennas) -> None:
    """Draws the pseudo-spectrum for Bluetooth direction finding"""
    antenna_response = response_vec(num_antennas, INCIDENT_ANGLE)
    sample_covariance = sample_cov(antenna_response)
    search_angles, signal_powers = music(sample_covariance)
    fig, ax = plt.subplots()
    ax.semilogy(search_angles, signal_powers)


interactive(children=(BoundedIntText(value=5, description='Antennas:', min=2), Output()), _dom_classes=('widge…